In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, LSTM, Lambda

from src.modeling.experiments.prototyping.lstm_protype_simple import (
                                                          set_tensorboard,
                                                          train_model,
                                                          evaluate_model,
                                                          validated_tf,
                                                          inference
                                                          )

from src.modeling.experiments.prototyping.lstm_simple_initial_training import (build_path, read_data_from_paths)

# Troubleshooting differnce in training loss between inital prototype and model objevts


In [ ]:
PATH_TO_DATA = '/projects/p31961/gaby_data/aggregated_data/data_pipeline_downsampled/datasets'

MODEL_ID = 'LSTM_03_simple_downsampled'
MODEL_PATH_TO_SAVE = '/projects/p31961/dopamine_modeling/results/models/'

TENSORBOARD_CALLBACK = set_tensorboard(MODEL_ID)

X_train_path = build_path(PATH_TO_DATA, 'X_train')
y_train_path = build_path(PATH_TO_DATA, 'y_train')
X_test_path = build_path(PATH_TO_DATA, 'X_test')
y_test_path = build_path(PATH_TO_DATA, 'y_test')

# # read data from parquet files
X_train = pd.read_parquet(X_train_path)
y_train = pd.read_parquet(y_train_path)
X_test = pd.read_parquet(X_test_path)
y_test = pd.read_parquet(y_test_path)

X_train = X_train[::500]
X_test = X_test[::500]
y_train = y_train[::500] 
y_test = y_test[::500]




In [ ]:

# # #build lodel
def experiment_01():
    
    def build_lstm(sequence_length, input_dimentions, units):

        input_shape = (sequence_length, input_dimentions)
        ltsm_model = Sequential([
            Lambda(lambda x: tf.expand_dims(x, axis=-1), input_shape=[None]),
            LSTM(units, input_shape=input_shape),
            Dense(1)
        ])
        return ltsm_model

    for units in range(32, 128, 32):
        model = build_lstm(sequence_length=90, input_dimentions=X_train.shape[1],units = units) 
        train_model(model, X_train, y_train, TENSORBOARD_CALLBACK)


In [ ]:
experiment_01()

In [ ]:
from src.models.SimpleLSTM import SimpleLSTM

def experiment_02():
    
    for units in range(32, 128, 32):
        model = SimpleLSTM(lstm_1_units = units, sequence_length=90, num_features=X_train.shape[1])
        model.compile(optimizer = 'adam', loss = 'mean_squared_error')
        model.fit(X_train, y_train, epochs = 50)
        
experiment_02()
        
    
    

In [ ]:
from src.models.StackedLSTM import StackedLSTM

def experiment_03():
    
    for units in range(32, 128, 32):
        model = StackedLSTM(lstm_1_units = units, sequence_length=90, num_features=X_train.shape[1])
        model.compile(optimizer = 'adam', loss = 'mean_squared_error')
        model.fit(X_train, y_train, epochs = 50)
        
experiment_03()

# RESOLVED: I had the wrong activation in the dense output for both Simple and StackLSTM. Was relu, is now default 'None- so linear is applied'